In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [2]:
loans_df = pd.read_csv('./data/lending-club-data.csv',low_memory=False)

loans_df.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,sub_grade_num,delinq_2yrs_zero,pub_rec_zero,collections_12_mths_zero,short_emp,payment_inc_ratio,final_d,last_delinq_none,last_record_none,last_major_derog_none
0,1077501,1296599,5000,5000,4975,36 months,10.65,162.87,B,B2,...,0.4,1.0,1.0,1.0,0,8.14350,20141201T000000,1,1,1
1,1077430,1314167,2500,2500,2500,60 months,15.27,59.83,C,C4,...,0.8,1.0,1.0,1.0,1,2.39320,20161201T000000,1,1,1
2,1077175,1313524,2400,2400,2400,36 months,15.96,84.33,C,C5,...,1.0,1.0,1.0,1.0,0,8.25955,20141201T000000,1,1,1
3,1076863,1277178,10000,10000,10000,36 months,13.49,339.31,C,C1,...,0.2,1.0,1.0,1.0,0,8.27585,20141201T000000,0,1,1
4,1075269,1311441,5000,5000,5000,36 months,7.90,156.46,A,A4,...,0.8,1.0,1.0,1.0,0,5.21533,20141201T000000,1,1,1


In [3]:
loans_df['safe_loans'] = loans_df['bad_loans'].apply(lambda cell_val : +1 if cell_val == 0 else -1)
loans_df.drop(columns=['bad_loans'])

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,delinq_2yrs_zero,pub_rec_zero,collections_12_mths_zero,short_emp,payment_inc_ratio,final_d,last_delinq_none,last_record_none,last_major_derog_none,safe_loans
0,1077501,1296599,5000,5000,4975,36 months,10.65,162.87,B,B2,...,1.0,1.0,1.0,0,8.143500,20141201T000000,1,1,1,1
1,1077430,1314167,2500,2500,2500,60 months,15.27,59.83,C,C4,...,1.0,1.0,1.0,1,2.393200,20161201T000000,1,1,1,-1
2,1077175,1313524,2400,2400,2400,36 months,15.96,84.33,C,C5,...,1.0,1.0,1.0,0,8.259550,20141201T000000,1,1,1,1
3,1076863,1277178,10000,10000,10000,36 months,13.49,339.31,C,C1,...,1.0,1.0,1.0,0,8.275850,20141201T000000,0,1,1,1
4,1075269,1311441,5000,5000,5000,36 months,7.90,156.46,A,A4,...,1.0,1.0,1.0,0,5.215330,20141201T000000,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122602,9856168,11708132,6000,6000,6000,60 months,23.40,170.53,E,E5,...,0.0,1.0,1.0,1,4.487630,20190101T000000,0,1,0,-1
122603,9795013,11647121,15250,15250,15250,36 months,17.57,548.05,D,D2,...,0.0,0.0,1.0,0,10.117800,20170101T000000,0,0,0,1
122604,9695736,11547808,8525,8525,8525,60 months,18.25,217.65,D,D3,...,0.0,1.0,1.0,0,6.958120,20190101T000000,0,1,0,-1
122605,9684700,11536848,22000,22000,22000,60 months,19.97,582.50,D,D5,...,1.0,0.0,1.0,0,8.961540,20190101T000000,1,0,1,-1


In [4]:
target = 'safe_loans'

features = ['grade',                     # grade of the loan (categorical)
            'sub_grade_num',             # sub-grade of the loan as a number from 0 to 1
            'short_emp',                 # one year or less of employment
            'emp_length_num',            # number of years of employment
            'home_ownership',            # home_ownership status: own, mortgage or rent
            'dti',                       # debt to income ratio
            'purpose',                   # the purpose of the loan
            'payment_inc_ratio',         # ratio of the monthly payment to income
            'delinq_2yrs',               # number of delinquincies 
            'delinq_2yrs_zero',          # no delinquincies in last 2 years
            'inq_last_6mths',            # number of creditor inquiries in last 6 months
            'last_delinq_none',          # has borrower had a delinquincy
            'last_major_derog_none',     # has borrower had 90 day or worse rating
            'open_acc',                  # number of open credit accounts
            'pub_rec',                   # number of derogatory public records
            'pub_rec_zero',              # no derogatory public records
            'revol_util',                # percent of available credit being used
            'total_rec_late_fee',        # total late fees received to day
            'int_rate',                  # interest rate of the loan
            'total_rec_int',             # interest received to date
            'annual_inc',                # annual income of borrower
            'funded_amnt',               # amount committed to the loan
            'funded_amnt_inv',           # amount committed by investors for the loan
            'installment',               # monthly payment owed by the borrower
            ]

In [5]:
n = loans_df.shape[0]
loans_df = loans_df[features + [target]].dropna()
na=loans_df.shape[0]
print(f" {n - na =}, {na}")

 n - na =29, 122578


In [6]:
safe_loans_raw = loans_df[loans_df[target] == +1]
risky_loans_raw = loans_df[loans_df[target] == -1]
print(f"{safe_loans_raw.shape =}, {risky_loans_raw.shape =}")

safe_loans_raw.shape =(99431, 25), risky_loans_raw.shape =(23147, 25)


In [7]:
risky_loans = risky_loans_raw.copy()
safe_loans = safe_loans_raw.sample(frac=risky_loans_raw.shape[0]/safe_loans_raw.shape[0],random_state=1)

loans_data =risky_loans._append(safe_loans)

In [8]:
print(f"{safe_loans.shape[0]}, {loans_data.shape[0]}")

23147, 46294


In [9]:
print(loans_data.dtypes)
categorical_variables = list(loans_data.select_dtypes(include=['object']).columns)
print(categorical_variables)

one_hot_data = pd.get_dummies(loans_data[categorical_variables], prefix=categorical_variables)
# need to add inplace in oreder to drop columns.
loans_data.drop(columns=categorical_variables, axis=1, inplace=True)
loans_data = pd.concat([loans_data, one_hot_data], axis=1)

grade                     object
sub_grade_num            float64
short_emp                  int64
emp_length_num             int64
home_ownership            object
dti                      float64
purpose                   object
payment_inc_ratio        float64
delinq_2yrs              float64
delinq_2yrs_zero         float64
inq_last_6mths           float64
last_delinq_none           int64
last_major_derog_none      int64
open_acc                 float64
pub_rec                  float64
pub_rec_zero             float64
revol_util               float64
total_rec_late_fee       float64
int_rate                 float64
total_rec_int            float64
annual_inc               float64
funded_amnt                int64
funded_amnt_inv            int64
installment              float64
safe_loans                 int64
dtype: object
['grade', 'home_ownership', 'purpose']


In [10]:
np.random.seed(1)

train_data,validation_data = train_test_split(loans_data,test_size=.2)

In [11]:
from sklearn.ensemble import GradientBoostingClassifier

model_5 = GradientBoostingClassifier(n_estimators=5,max_depth=6,random_state=0)

X = train_data.loc[:,train_data.columns != target].values
y=train_data[target].values



In [12]:
model_5.fit(X,y)

GradientBoostingClassifier(max_depth=6, n_estimators=5, random_state=0)

In [13]:
#all +ve and -ve
validation_safe_loans = validation_data[validation_data[target] == 1]
validation_risky_loans = validation_data[validation_data[target] == -1]

#take 2 samples 
sample_validation_data_risky = validation_risky_loans[0:2]
sample_validation_data_safe = validation_safe_loans[0:2]

#Merge samples
sample_validation_data = sample_validation_data_safe._append(sample_validation_data_risky)
sample_validation_data[target]


93539     1
10285     1
121325   -1
74825    -1
Name: safe_loans, dtype: int64

In [14]:
sample_validation_data.loc[:,train_data.columns != target]

,sub_grade_num,short_emp,emp_length_num,dti,payment_inc_ratio,delinq_2yrs,delinq_2yrs_zero,inq_last_6mths,last_delinq_none,last_major_derog_none,...,purpose_debt_consolidation,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_small_business,purpose_vacation,purpose_wedding
93539,0.6,0,5,31.70,13.30900,0.0,1.0,0.0,0,1,...,True,False,False,False,False,False,False,False,False,False
10285,0.6,0,4,10.80,8.19823,0.0,1.0,0.0,1,1,...,True,False,False,False,False,False,False,False,False,False
121325,0.4,0,6,23.57,6.50943,0.0,1.0,1.0,1,1,...,True,False,False,False,False,False,False,False,False,False
74825,1.0,0,7,18.56,19.08670,0.0,1.0,0.0,1,1,...,True,False,False,False,False,False,False,False,False,False


In [15]:
print(model_5.predict(sample_validation_data.loc[:,train_data.columns != target].values))
print(model_5.predict_proba(sample_validation_data.loc[:,train_data.columns != target].values))

[-1  1 -1 -1]
[[0.54974932 0.45025068]
 [0.41588311 0.58411689]
 [0.6787601  0.3212399 ]
 [0.5564696  0.4435304 ]]


![title](./img/ensem-1.png "ShowMyImage")

In [16]:
val_X = validation_data.loc[:,train_data.columns != target].values
val_y = validation_data[target].values
print(val_y)
print(model_5.score(val_X,val_y))
predictions = model_5.predict(val_X)
print(predictions)

[-1  1 -1 ...  1 -1 -1]
0.6595744680851063
[-1 -1 -1 ...  1 -1 -1]


In [17]:
neg=val_y== -1
pos=val_y ==1

true_positives = np.sum(val_y[pos] == predictions[pos])
true_negatives = np.sum(val_y[neg] == predictions[neg])
false_positives = np.sum(predictions[neg] != val_y[neg])
false_negatives=np.sum(predictions[pos] != val_y[pos])

print(f"{false_positives =}, {false_negatives=}, {true_positives=},{true_negatives=}, {len(val_y)}")


false_positives =1673, false_negatives=1479, true_positives=3147,true_negatives=2960, 9259


In [18]:
validation_data['predictions'] = model_5.predict_proba(val_X)[:, 1]
validation_data.sort_values("predictions", ascending=False, inplace=True)

In [19]:
print (validation_data['predictions'][:5])

29080     0.66411
54108     0.66411
9624      0.66411
85067     0.66411
116768    0.66411
Name: predictions, dtype: float64


In [20]:
# YOUR CODE HERE
print (validation_data[:5].loc[:, ['grade_A', 'grade_B', 'grade_C', 'grade_D', 'grade_E', 'grade_F', 'grade_G', 'predictions', target]])

low_safe = validation_data.sort_values('predictions', ascending=True)
print (low_safe[:5].loc[:, ['grade_A', 'grade_B', 'grade_C', 'grade_D', 'grade_E', 'grade_F', 'grade_G', 'predictions', target]])


        grade_A  grade_B  grade_C  grade_D  grade_E  grade_F  grade_G  \
29080      True    False    False    False    False    False    False   
54108      True    False    False    False    False    False    False   
9624       True    False    False    False    False    False    False   
85067      True    False    False    False    False    False    False   
116768     True    False    False    False    False    False    False   

        predictions  safe_loans  
29080       0.66411           1  
54108       0.66411           1  
9624        0.66411           1  
85067       0.66411          -1  
116768      0.66411           1  
       grade_A  grade_B  grade_C  grade_D  grade_E  grade_F  grade_G  \
30677     True    False    False    False    False    False    False   
52678    False     True    False    False    False    False    False   
25697    False    False     True    False    False    False    False   
24272    False     True    False    False    False    False    False 

In [21]:


model_10 = GradientBoostingClassifier(n_estimators=10, max_depth=6, random_state=0).fit(X, y)
model_50 = GradientBoostingClassifier(n_estimators=50, max_depth=6, random_state=0).fit(X, y)
model_100 = GradientBoostingClassifier(n_estimators=100, max_depth=6, random_state=0).fit(X, y)
model_200 = GradientBoostingClassifier(n_estimators=200, max_depth=6, random_state=0).fit(X, y)
model_500 = GradientBoostingClassifier(n_estimators=500, max_depth=6, random_state=0).fit(X, y)



In [17]:


print('Train accuraccy {}, val accuracy {}'.format(model_10.score(X, y), model_10.score(val_X, val_y)))
print('Train accuraccy {}, val accuracy {}'.format(model_50.score(X, y), model_50.score(val_X, val_y)))
print('Train accuraccy {}, val accuracy {}'.format(model_100.score(X, y), model_100.score(val_X, val_y)))
print('Train accuraccy {}, val accuracy {}'.format(model_200.score(X, y), model_200.score(val_X, val_y)))
print('Train accuraccy {}, val accuracy {}'.format(model_500.score(X, y), model_500.score(val_X, val_y)))

